In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =======

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create session-based features with [NVTabular](https://github.com/NVIDIA/NVTabular) to train a session-based recommendation model for next item prediction task.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

- Import required libraries

In [2]:
import os
import glob

import torch 
import numpy

import cudf
import cupy as cp
import nvtabular as nvt

- Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = 100000
inputs = {
    'session_id': numpy.random.randint(70000, 80000, NUM_ROWS),
    'day': numpy.random.randint(1, 10, NUM_ROWS),
    'item_id': numpy.random.randint(0, 51196, NUM_ROWS),
    'category': numpy.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': numpy.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : numpy.random.uniform(0, 1, NUM_ROWS),
    }
df = cudf.DataFrame(inputs)
df.head()

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin
0,77789,4,18449,255,0.929688,0.896908
1,78472,3,36495,124,0.040510,0.800594
2,70526,1,44429,330,0.754144,0.301041
3,78123,4,21531,177,0.048006,0.983697
4,74701,5,44893,132,0.540213,0.688128


In [5]:
# 'item_id': numpy.random.lognormal(3., 1., NUM_ROWS).astype(numpy.int32),

## Feature Engineering with NVTabular

Deep Learning models require the input feature in a specific format. Categorical features needs to be continuous integers (0, ..., |C|) to use them with an embedding layer. We will use NVTabular to preprocess the categorical features, so that all categorical columns will be encoded to continuous integers. Note that we also add `1` after we categorify the categorical columns, the reason for that we want the encoded null values to start from `1` instead of `0` because we reserve `0` for padding the seqeunce features.

Here our goal is to create session-based features.  In this cell, we are creating temporal features and grouping them together at the session level, sorting the interactions by time. Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features in a couple of lines on a GPU.

In [6]:
# Categorify categorical features

categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify() >> nvt.ops.LambdaOp(lambda col: col +1)
# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'timestamp/age_days', 'timestamp/weekday/sin']

groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")

groupby_features_trim = (groupby_features['category-list', 'item_id-list', 'timestamp/age_days-list', 'timestamp/weekday/sin-list']) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

MINIMUM_SESSION_LENGTH = 2
selected_features = groupby_features['item_id-count', 'day-first', 'session_id'] + groupby_features_trim\

filtered_sessions = (selected_features) >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)\

workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(df, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

In [7]:
sessions_gdf.head(3)

,item_id-count,day-first,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,24,3,2,"[237, 89, 40, 13, 90, 152, 186, 52, 216, 203, ...","[8478, 12927, 33021, 9892, 17736, 933, 14584, ...","[0.988693451346936, 0.21186240503101517, 0.455...","[0.9927485451479546, 0.8897108642370064, 0.437..."
1,24,9,3,"[296, 16, 327, 116, 274, 318, 326, 106, 126, 8...","[25350, 41066, 32857, 5772, 33616, 4193, 1247,...","[0.9268431688931404, 0.3627724217947451, 0.377...","[0.13003177193725124, 0.8991020624389696, 0.85..."
2,23,4,4,"[99, 33, 134, 117, 101, 73, 211, 142, 222, 295...","[15467, 10213, 15297, 9653, 6386, 22949, 14638...","[0.854926921498967, 0.6829013678893918, 0.7060...","[0.9509296729881792, 0.07915906890287194, 0.43..."


- we can save our workflow

In [8]:
workflow.save('workflow')

## Export pre-processed data by day¶

In [9]:
OUTPUT_FOLDER = os.environ.get("OUTPUT_FOLDER", INPUT_DATA_DIR + "sessions_by_day")
!mkdir -p $OUTPUT_FOLDER

In [10]:
# requires cudf + cupy + nvtabular + dask_cudf
from transformers4rec.utils.gpu_preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_FOLDER,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 12.03it/s]


## Checking the pre-processing outputs¶

In [11]:
TRAIN_PATHS = sorted(glob.glob(os.path.join(OUTPUT_FOLDER, "1", "train.parquet")))

In [12]:
gdf = cudf.read_parquet(TRAIN_PATHS[0])
gdf.head()

,item_id-count,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,22,7,"[179, 127, 246, 260, 243, 135, 209, 274, 276, ...","[26443, 8508, 25311, 2680, 10604, 27137, 14466...","[0.9179285436064166, 0.019458452311733376, 0.1...","[0.36535143583698426, 0.7867226414482792, 0.56..."
1,21,14,"[221, 281, 267, 320, 318, 63, 23, 183, 273, 14...","[2345, 40166, 43805, 3805, 1331, 13600, 27337,...","[0.1799245072294352, 0.2610959321798335, 0.319...","[0.6439663175170048, 0.06590634508556537, 0.56..."
2,21,21,"[253, 83, 55, 219, 306, 258, 116, 186, 325, 28...","[23464, 8572, 17526, 5089, 27877, 27640, 14136...","[0.4801740006472296, 0.8808313800410272, 0.119...","[0.49752319798156075, 0.18493971673580478, 0.4..."
3,20,41,"[142, 255, 160, 231, 285, 129, 267, 174, 24, 2...","[235, 25040, 3455, 15740, 307, 8342, 34813, 58...","[0.5249228446980618, 0.023509455710684635, 0.1...","[0.38186513715524184, 0.37479180301121706, 0.5..."
4,19,67,"[190, 250, 190, 121, 76, 172, 253, 143, 203, 1...","[11556, 10261, 42204, 12966, 585, 9622, 22766,...","[0.25153120929918804, 0.5920753070512191, 0.04...","[0.009368517687431743, 0.1183022455447269, 0.7..."
